In [ ]:
import scanpy as sc
import anndata as ad
import gseapy as gp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [2]:
sc.settings.set_figure_params(dpi=200,dpi_save=500,vector_friendly=False,facecolor="white")

In [3]:
colors = [(0, '#D3D3D3'), (1, '#FF9676')]  
n_bins = 100  
cmap_name = 'gray2orange'
cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

In [4]:
colors = [(0, '#D3D3D3'), (1, '#097B58')]  
n_bins = 100 
cmap_name = 'gray2green'
cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

In [5]:
adata = sc.read_mtx('path/to/your/file.mtx')

In [6]:
adata = ad.AnnData.transpose(adata)

In [7]:
gene_data = pd.read_csv('/vsphhome/fengguoqing/lung_geneNames_upload.csv',header=None)
cell_data = pd.read_csv('/vsphhome/fengguoqing/lung_cellNames.csv',header=None)

In [ ]:
adata.X

In [9]:
adata.obs_names = cell_data[0]
adata.var_names = gene_data[0]

In [ ]:
adata

In [11]:
umap_cor = pd.read_csv('/vsphhome/fengguoqing/lung_clusterfile.txt',sep='\t')

In [12]:
meta_data = pd.read_csv('/vsphhome/fengguoqing/lung_metaData.txt',dtype=str,sep='\t')

The first cell was removed because it lacked metadata information.

In [13]:
adata = adata[1:,:]

In [14]:
umap_data = umap_cor[1:]

In [15]:
meta_data2 = meta_data[1:]

In [16]:
meta_data2.set_index('NAME',inplace=True)

In [17]:
meta_data2 = meta_data2.reindex(adata.obs_names)

In [18]:
adata.obs = meta_data2

In [19]:
umap_data.set_index('NAME',inplace=True)

In [20]:
umap_data = umap_data.reindex(adata.obs_names)

In [21]:
umap_data = umap_data.astype(float)

In [22]:
adata.obsm['X_umap'] = umap_data.values

In [23]:
marker_gene = ['RMC1','NPC1']

In [ ]:
adata

In [25]:
adata.obs['total_counts'] = adata.X.sum(axis=1)
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes", "total_counts"],
    jitter=0.4,
    multi_panel=True,
    stripplot=False
)

In [27]:
adata_control = adata[adata.obs['group']=='Control',:]
adata_covid = adata[adata.obs['group']=='COVID-19',:]

In [29]:
adata_control,adata_covid

(View of AnnData object with n_obs × n_vars = 36677 × 34546
     obs: 'biosample_id', 'donor_id', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'group', 'cell_type_main', 'cell_type_intermediate', 'cell_type_fine', 'initial_clustering', 'age', 'sex', 'recorded_race', 'recorded_ethnicity', 'intubation_days', 'interval_death_symptoms_onset_days', 'pmi_h', 'total_counts', 'n_genes'
     obsm: 'X_umap',
 View of AnnData object with n_obs × n_vars = 79636 × 34546
     obs: 'biosample_id', 'donor_id', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'group', 'cell_type_main', 'cell_type_intermediate', 'cell_type_fine', 'initial_clustering', 'age', 'sex', 'recorded_race', 'recorded_ethnicity', 'intubation_days', 

In [ ]:
sc.pl.umap(
    adata_control,
    color="cell_type_intermediate",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=8,
    legend_fontoutline=1
)

In [ ]:
sc.pl.umap(adata_control, color=['NPC1'], color_map=cmap2, show=False)
plt.savefig('./dotplot_control_npc1.png',dpi=800)
plt.savefig('./umap_control_npc1.svg',dpi=800)

In [ ]:
sc.pl.umap(adata_control, color=['RMC1'], color_map=cmap1, show=False)
plt.savefig('./dotplot_control_rmc1.png',dpi=800)
plt.savefig('./umap_control_rmc1.svg',dpi=800)

In [ ]:
sc.pl.dotplot(adata_control, ['RMC1'], "cell_type_intermediate", cmap=cmap1, show=False)
plt.savefig('./dotplot_control_rmc1.png',dpi=800)
plt.savefig('./dotplot_control_rmc1.pdf')

In [ ]:
sc.pl.dotplot(adata_control, ['NPC1','RMC1'], "cell_type_intermediate", cmap=cmap2, show=False)
plt.savefig('./dotplot_control_all.png',dpi=800)
plt.savefig('./dotplot_control_all.pdf')

In [ ]:
sc.pl.umap(
    adata_covid,
    color="cell_type_intermediate",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=6,
    legend_fontoutline=1,
    show=False,
    title = ''
)
plt.savefig('UMAPcovid_plot.png', format='png', dpi=800)


sc.pl.umap(adata_covid, color=['RMC1'],color_map=cmap1,show=False)
plt.savefig('feature_covid_rmc1.png', format='png', dpi=800)

sc.pl.umap(adata_covid, color=['NPC1'],color_map=cmap2,show=False)
plt.savefig('feature_covid_npc1.png', format='png', dpi=800)

sc.pl.dotplot(adata_covid, marker_gene, "cell_type_intermediate", cmap=cmap1,show=False)
plt.savefig('dot_covid1.pdf', format='pdf')

sc.pl.dotplot(adata_covid, marker_gene, "cell_type_intermediate", cmap=cmap2,show=False)
plt.savefig('dot_covid2.pdf', format='pdf')

In [ ]:
sc.pl.umap(
    adata_covid,
    color="cell_type_intermediate",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=3,
    legend_fontoutline=1,
    show=False
)
plt.savefig('./UMAPcovid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color='cell_type_intermediate',legend_fontsize=3,show=False)
plt.savefig('./UMAP2covid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color=marker_gene,color_map=cmap,show=False)
plt.savefig('./feature_covid.pdf', format='pdf')


sc.pl.dotplot(adata_covid, marker_gene, "cell_type_intermediate", cmap=cmap,show=False)
plt.savefig('./dot_covid.pdf', format='pdf')

In [ ]:
sc.pl.umap(
    adata_covid,
    color="cell_type_main",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=3,
    legend_fontoutline=1,
    show=False
)
plt.savefig('./1UMAPcovid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color='cell_type_main',legend_fontsize=3,show=False)
plt.savefig('./1UMAP2covid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color=marker_gene,color_map=cmap,show=False)
plt.savefig('./1feature_covid.pdf', format='pdf')


sc.pl.dotplot(adata_covid, marker_gene, "cell_type_main", cmap=cmap,show=False)
plt.savefig('./1dot_covid.pdf', format='pdf')

In [ ]:
sc.pl.umap(
    adata_covid,
    color="cell_type_fine",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=3,
    legend_fontoutline=1,
    show=False
)
plt.savefig('./2UMAPcovid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color='cell_type_fine',legend_fontsize=3,show=False)
plt.savefig('./2UMAP2covid_plot.pdf', format='pdf')


sc.pl.umap(adata_covid, color=marker_gene,color_map=cmap,show=False)
plt.savefig('./2feature_covid.pdf', format='pdf')


sc.pl.dotplot(adata_covid, marker_gene, "cell_type_fine", cmap=cmap,show=False)
plt.savefig('./2dot_covid.pdf', format='pdf')